# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# Corrected and completed Gradio implementation for chat interface
import gradio as gr
import requests
import json
import os
import openai
import anthropic
import google.generativeai
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv

In [2]:
# Load environment variables
load_dotenv(override=True)

True

In [3]:
# Configure API clients
openai_client = openai.OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()
OLLAMA_API = os.getenv('OLLAMA_API', "http://localhost:11434/api/chat")

In [4]:

# Mapping user-friendly model names to actual model identifiers
MODEL_MAP = {
    'GPT': 'gpt-4o-mini',
    'Claude': 'claude-3-haiku-20240307',
    'Gemini': 'gemini-1.5-flash',
    'Ollama': 'llama3.2'
}

In [5]:
# System prompt to guide the model behavior
system_prompt = """You are a helpful assistant of Quadrasoft CRM helping users to solve technical issues.
If User asks about GPS coordinates that they cannot be found your answer must look like: Please check whether your GPS is on.
Then ask user to check in the tablet or phone settings that you have switched both sources of GPS coordinates.
Offer User to switch on 'any accuracy' button.
If User says that GPS coordinates are not accurate, please advise him to continue work.
Ask him the organization name and address to send the request for coordinates check.
If and only if the user after assistant question sends the request, say thank you and inform that task to check coordinates was created.
If user asks other things, ask him again what is the problem.
If user answers, tell him that his request has been received and wait for the operator.
The user will write in Russian or Ukrainian - answer in the language he is writing."""

In [6]:
# Function to stream responses from Ollama
# Streams delta chunks and yields them
def stream_ollama(messages, model):
    payload = {"model": model, "messages": messages, "stream": True}
    response = requests.post(OLLAMA_API, json=payload, stream=True)
    for line in response.iter_lines():
        if line:
            json_data = json.loads(line.decode('utf-8'))
            if 'message' in json_data:
                yield json_data['message']['content']

In [7]:
# Function to stream responses from Google Gemini
# Streams delta chunks and yields them
def stream_gemini(messages, model):
    model_gemini = google.generativeai.GenerativeModel(model_name=model, system_instruction=system_prompt)
    stream = model_gemini.generate_content(messages[-1]['content'], stream=True)
    for chunk in stream:
        if chunk.text:
            yield chunk.text

In [8]:
# Claude – streaming with Anthropic SDK v1

def stream_claude(messages, model):
    # Convert messages to the format expected by Claude
    claude_messages = []
    for msg in messages:
        if msg["role"] == "system":
            continue  # System message is handled separately
        claude_messages.append({
            "role": "user" if msg["role"] == "user" else "assistant",
            "content": msg["content"]
        })
    
    # Make a non-streaming request as a fallback approach
    try:
        response = claude.messages.create(
            model=model,
            system=system_prompt,
            messages=claude_messages,
            max_tokens=1000
        )
        # Just yield the full response at once since streaming isn't working
        yield response.content[0].text
    except Exception as e:
        yield f"Error: {str(e)}"


In [9]:
# Function to stream responses from OpenAI GPT
# Streams delta chunks and yields them
def stream_gpt(messages, model):
    stream = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content:
            yield content

In [10]:
# Dispatcher dictionary mapping model names to streaming functions
STREAM_FUNCS = {
    "GPT": stream_gpt,
    "Claude": stream_claude,
    "Gemini": stream_gemini,
    "Ollama": stream_ollama
}

In [11]:
def chat(user_message, model_choice, history):
    # Start from the system prompt
    messages = [{"role": "system", "content": system_prompt}] + history

    # Add new user message
    messages.append({"role": "user", "content": user_message})

    # Stream response
    full_response = ""
    response_generator = STREAM_FUNCS[model_choice](messages, MODEL_MAP[model_choice])

    for chunk in response_generator:
        full_response += chunk
        # Important: Update the chat history with both the user message and assistant reply
        updated_history = history + [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": full_response}
        ]
        yield updated_history


In [22]:
with gr.Blocks(theme=gr.themes.Ocean()) as interface:


    gr.Markdown("# 📄 Підтримка компанії Квадрасофт")

    model_selector = gr.Dropdown(["GPT", "Claude", "Gemini", "Ollama"], label="Select Model", value="Gemini")
    chatbot = gr.Chatbot(height=300, type='messages')
    msg = gr.Textbox(placeholder="Ваше питання...", label="Повідомлення")
    clear = gr.Button("Очистити")

    def submit(user_message, model_choice, chat_history):
        for updated_history in chat(user_message, model_choice, chat_history):
            pass
        return updated_history

    msg.submit(submit, [msg, model_selector, chatbot], chatbot)
    clear.click(lambda: [], None, chatbot, queue=False)

    gr.Examples([
        ["Як оновити координати?", "Gemini"],
        ["Чому у мене невірно відображені координати?", "Claude"],
        ["GPS координати не знайдені", "GPT"]
    ], inputs=[msg, model_selector])



 


In [23]:
# Launch the interface
interface.launch(pwa=True, share=True)

* Running on local URL:  http://127.0.0.1:7869

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
